Copyright (c) 2020 Martin Holle. Alle Rechte vorbehalten. Lizensiert unter der MIT-Lizenz.

# Covid-19 Statistics Aachen: Datenabfrage

Abfrage der Daten von der Website der Städteregion Aachen und Speichern in einer Excel-Datei für die Datenübergabe an den nächsten Schritt, in dem die Daten aufbereitet werden.

## Vorbereitungen

- Benötigte Imports
- Konfiguration aus zentraler `.ini`-Datei einlesen
- Konfiguration und Instanzierung des Loggers
- Globale Variablen definieren

In [24]:
import pandas as pd
import numpy as np

import re
from datetime import date
from datetime import timedelta
from datetime import date
from datetime import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup

import logging
import configparser

# Konfiguration einlesen
config = configparser.ConfigParser()
config.read('config.ini')

# Konfiguration des Loggings
# - Die Logging-Ausgaben werden in der lokalen Datei covid-19-datenabfrage.log geschrieben
# - Für die Ausgabe wird eine bestimmte Formatierung konfiguriert
fhandler = logging.FileHandler(filename=config['Logging']['LogFileName'], mode='a')

# TODO: Formatierung finalisieren (Tausendstel-Sekunden, Tag des Monats, 1. Zeichen des Levels)
formatter = logging.Formatter('%(asctime)s %(levelname)-1.1s %(name)-20.20s - %(message)s')
fhandler.setFormatter(formatter)

# Instanzierung und Konfigurierung des Loggers
log = logging.getLogger("datenabfrage")
log.addHandler(fhandler)
log.setLevel(logging.DEBUG)

# Für die Zwischenspeicherung des eingelesenen HTML
# - Wenn die Website in der aktuellen Session schon einmal abgefragt wurde, wird das Ergebnis
#   der Abfrage in dieser Variablen gesichert
# - Dies erleichtert die Entwicklung der nachfolgenden Verabeitungsschritte und führt nicht immer wieder zu
#   neuen überflüssigen Abfragen der Website
html_payload = None

## Einlesen der existierenden Excel-Datei

- Datei und Seite der Excel-Datei: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **C**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **D**: Anzahl neuer Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **E**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **F**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spalte A als Datum interpretieren
- Die erste Zeile (Header) überspringen
- Label der Spalten explizit setzen

In [25]:
col_names = ['Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]

try:
    c19_cases = pd.read_excel(config['Rohdaten']['FileName'], 
                              sheet_name=config['Rohdaten']['SheetName'], 
                              index_col=0,
                              parse_dates=[0],
                              skiprows=[],
                              names=col_names)
except FileNotFoundError as err: 
    log.warning('Error during pd.read_excel(): {0}'.format(err))
    # Leere DataFrame für den Start erzeugen
    c19_cases = pd.DataFrame(columns=col_names, index=pd.DatetimeIndex([], name='Datum'))
    
c19_cases.tail(14)

,Uhrzeit,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
Datum,,,,,,
2020-07-08,09:30:00,2010,999,99,1900,11
2020-07-10,09:30:00,2011,999,100,1908,3
2020-07-13,10:00:00,2021,1002,100,1908,13
2020-07-15,09:30:00,2025,1004,100,1909,16
2020-07-17,09:45:00,2030,1007,100,1917,13
2020-07-20,09:30:00,2037,1008,100,1917,20
2020-07-22,09:30:00,2044,1010,100,1920,24
2020-07-24,09:30:00,2058,1014,100,1922,36
2020-07-27,09:45:00,2067,1016,100,1928,39


## Datenabfrage

### Funktionen für die Datenabfrage und Extraktion der Rohtexte

- `robot_access_allowed()` - Via robots.txt prüfen, ob die Website durch ein Skript abgefragt und verabeitet werden darf
- `gather_html()` - Website abfragen und HTML zurückliefern
- `gather_text()` - Relevante Texte aus dem von der Website geliefertem HTML extrahieren und zurückliefern

*<u>Anmerkungen:</u>  
Die Website der Städteregion Aachen verwendet keine `robots.txt`, damit wäre die Funktion `robot_access_allowed()` eigentlich überflüssig. Hier ist sie nur der Vollständigkeit halber definiert und nicht ausimplementiert.*

*Leider unterscheidet sich die HTML-Struktur der aktuellen Meldungen von der des Meldungsarchivs so stark, dass schon an dieser Stelle eine Differenzierung aufgrund des Textes und nicht nur der HTML-Struktur erfolgen muss. Daher wurde das Argument `kind` zur Funktion `gather_html()` hinzugefügt.*

In [26]:
default_user_agent = config['Rohdaten']['UserAgent']

def robot_access_allowed(url: str, user_agent: str=default_user_agent) -> bool:
    """Request robots.txt. If exists, parse robots.txt and return True if scraping by this script is allowed."""
    
    log.debug("access_allowed(" + url + ", " + user_agent + ")")
    
    headers = { 'user-agent': user_agent }
    
    # Extract root of the website's URL
    
    # Request robots.txt from root of the website
    
    # If robots.txt exist, parse content of the file
    
    return True

def gather_html(url: str, user_agent: str=default_user_agent) -> str:
    """Request Website from <url> and return the HTML delivered by the Website as text."""

    log.debug("gather_html(" + url + ", " + user_agent + ")")
    
    # Website abfragen
    headers = { 'user-agent': user_agent }
    page = requests.get(url)
    log.debug("gather_html/page.status_code: %d", page.status_code)
    for key, val in page.headers.items():
        log.debug("gather_html/page.header(%s): %s", key, val)   
    if page.status_code == requests.codes.ALL_OK: # Alles ok
        return page.text
        
    return None

def gather_text(html_text: str) -> []:
    """
    Extract relevant text content from <html_text> and return dictionary with extracted text for 'Header', 'Abstract', 'Main'.
    """

    log.debug(f"gather_text(%d chars)", len(html_text))

    records = []
       
    # Parser instanzieren
    soup = BeautifulSoup(html_text, 'html.parser')

    # Relevante Objekte aus geliefertem HTML extrahieren
    # - Header
    # - Abstract (existiert nur für die aktuellen Meldungen, nicht im Meldungsarchiv)
    # - Main
    divs = soup.select('div.mid-col article > div.textcontent > div')
    for div in divs:
        # Header erkennen: Nun wenn dieser gefunden wird, macht es Sinn, nachfolgend nach 
        # Abstract und Haupttext zu suchen
        header = div.select('h2')
        if header:
            # Header: Text extrahieren
            header_text = next(header[0].stripped_strings)

            # Abstract: Text extrahieren
            abstract = div.select('div > div > ul > li')
            abstract_text = ''
            for a in abstract:
                for tx in a.stripped_strings:
                    abstract_text += tx
            
            # Haupttext extrahieren
            main = div.select('div > p')
            main_text = ''
            for tx in main[0].stripped_strings:
                main_text += tx

            # Neuen Eintrag mit den extahierten Texten hinzufügen 
            records.append({'Header': header_text, 'Abstract': abstract_text, 'Main': main_text })
                        
    log.debug("gather_text: %d records extracted from HTML", len(records))
    
    return records

### Funktionen für die Extraktion der Falldaten aus den Rohtexten

In der ersten Version erfolgt die Extraktion der Daten aus den Texten mit Hilfe regulärer Ausdrücke:

- Datum/Uhrzeit der Meldung wird aus dem `Header` extrahiert.
- Die eigentlichen Daten werden zweimal extrahiert: Einmal aus dem `Abstract` und zum zweiten Mal aus dem `Main`-Text. 
- Falls sowohl `Abstract` als auch `Main` eingelesen wurden, werden anschließend die extrahierten Daten miteinander verglichen. Nur wenn sie übereinstimmen, wird der Datensatz übernommen.

Die für das Parsen der eingelesenen Texte verwendeten regulären Ausdrücke versuchen einerseits, auf Nummer sicher zu gehen, um die richtigen Textstellen für das Einlesen der Zahlen zu treffen, und andererseits einige Freiheitsgrade zuzulassen: 

- Leerzeichen und andere "White Spaces" werden von Menschen hin und wieder vergesssen oder mehrfach eingegeben
- Textvariationen kommen vor, in denen einzelne Wörter nicht erscheinen oder hinzugefügt werden
- Manchmal werden einzelne Buchstaben weggelassen, ohne dass dies ein Schreibfehler wäre
- Und natürlich kommen auch Schreibfehler vor 

Die regulären Ausdrücke sind dadurch relativ unübersichtlich geworden und berücksichtigen dennoch nicht alle Situationen.

In [27]:
class NewsMeta():
    """Value class for meta data of news entry."""
    pass

class CaseFigures():
    """Value class for case figures, extracted from the news entries."""
    pass

def parse_header(header: str) -> NewsMeta:
    """
    Analyse <header> of the news and return date and time of news or None. 
    
    The <header> is expected to have the following format:
    
    'Aktuelle Lage Stadt und StädteRegion Aachen zum Corona-Virus; Montag, 22. Juni, 10:00 Uhr'
    
    Variables weekday name, date, and time will be extracted from header. Year assumed to be 2020.
    
    Parameters:
    ===========
    header - extracted raw text of header
    
    Returns:
    ========
    None - if no matching header found
    NewsMeta(weekday_name, day_of_month, month_name, hour, minutes, date, time, datetime)
    """
    
    monthnames = [ "Januar", "Februar", "März", "April", "Mai", "Juni", "Juli", "August", "September", 
                  "Oktober", "November", "Dezember" ]
    
    log.debug("parse_header(" + header[:40] + "...)")
    
    pattern = r"^Aktuelle Lage Stadt und StädteRegion Aachen zum Corona-Virus;\s*" + \
    r"(Montag|Dienstag|Mittwoch|Donnerstag|Freitag|Samstag|Sonntag),\s*" + \
    r"([0-9]{1,2})\.\s*(Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember),\s*" + \
    r"([0-9]{1,2}):([0-9]{2})\s*Uhr.*$"
        
    match = re.search(pattern, header)
    if match:
        meta = NewsMeta()
        try:
            meta.weekday_name = match.group(1)
            meta.day_of_month = int(match.group(2))
            meta.month_name = match.group(3)
            meta.hour = int(match.group(4))
            meta.minutes = int(match.group(5))
            meta.date = date(2020, monthnames.index(meta.month_name) + 1, meta.day_of_month)
            meta.time = time(meta.hour, meta.minutes)
            meta.datetime = datetime.combine(meta.date, meta.time)
            log.debug(f"parse_header/meta.weekday: %s", meta.weekday_name)        
            log.debug(f"parse_header/meta.day_of_month: %d", meta.day_of_month)        
            log.debug(f"parse_header/meta.month_name: %s", meta.month_name)        
            log.debug(f"parse_header/meta.hour: %d", meta.hour)        
            log.debug(f"parse_header/meta.minutes: %d", meta.minutes)        
            log.debug(f"parse_header/meta.date: %s", str(meta.date))        
            log.debug(f"parse_header/meta.time: %s", str(meta.time))
            log.debug(f"parse_header/meta.datetime: %s", str(meta.datetime))
            return meta
        except:
            log.warning("parse_header() failed parsing news header [1]")
            return None
    else: 
        log.warning("parse_header() failed parsing news header [2]")
        return None

def parse_cases(kind: str, text: str, pattern_total: str, pattern_recovered: str, pattern_deaths: str, 
                pattern_active: str) -> CaseFigures:
    """
    Analyse <abstract> or <main> text of the news and return Covid-19 case numbers included therein. 
  
    Parameters:
    ===========
    kind
    text
    pattern_total
    pattern_recovered
    pattern_deaths
    pattern_active
    
    Returns:
    ========
    None - if text could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
    """
    
    log.debug(f"parse_cases(%s, %s)", kind, text[:80])

    figures = CaseFigures()
    
    # Summe der Corona-Fälle extrahieren
    match = re.search(pattern_total, text)
    if match:
        try:
            figures.total = int(match.group(1))
            figures.total_AC = int(match.group(2))
            log.debug(f"parse_cases/figures.total: %d", figures.total)        
            log.debug(f"parse_cases/figures.total_AC: %d", figures.total_AC)        
        except:
            log.warning(f"parse_cases(%s) failed [1]", kind)
            return None
    else: 
        log.warning(f"parse_cases(%s) failed [2]", kind)
        return None
    
    # Summe der wieder Genesenen extrahieren
    match = re.search(pattern_recovered, text)
    if match:
        try:
            figures.recovered = int(match.group(1))
            log.debug(f"parse_cases/figures.recovered: %d", figures.recovered)        
        except:
            log.warning(f"parse_cases(%s) failed [3]", kind)
            return None
    else: 
        log.warning(f"parse_cases(%s) failed [4]", kind)
        return None
    
    # Summe der Corona-Toten extrahieren
    match = re.search(pattern_deaths, text)
    if match:
        try:
            figures.deaths = int(match.group(1))
            log.debug(f"parse_cases/figures.deaths: %d", figures.deaths)        
        except:
            log.warning(f"parse_cases(%s) failed [5]", kind)
            return None
    else: 
        log.warning(f"parse_cases(%s) failed [6]", kind)
        return None
    
    # Anzahl der aktiven Infektionen extrahieren
    match = re.search(pattern_active, text)
    if match:
        try:
            figures.active = int(match.group(1))
            log.debug(f"parse_cases/figures.active: %d", figures.active)        
        except:
            log.warning(f"parse_cases(%s) failed [7]", kind)
            return None
    else: 
        log.warning(f"parse_cases(%s) failed [8]: %s", kind, text)
        return None

    return figures

def parse_abstract(abstract: str) -> CaseFigures:
    """
    Analyse <abstract> of the news and return Covid-19 case numbers included in abstract. 

    The relevant area of the <abstract> is expected to have the following format:
    
    'Aktuell 1999 bestätigte Coronafälle in der StädteRegion Aachen (davon 994 in der Stadt Aachen). '
    '1880 ehemals positiv auf das Corona-Virus getestete Personen sind inzwischen wieder gesund. '
    'Bislang 98 Todesfälle. Damit aktuell 21 nachgewiesene Infizierte.'
    
    Values will be extracted from abstract text. 
    
    Parameters:
    ===========
    abstract - extracted raw text of abstract
    
    Returns:
    ========
    None - if abstract could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
    """
    
    log.debug("parse_abstract(" + abstract[:40] + "...)")
    
    pattern_total = r"^Aktuell\s*([0-9]{1,6})\s*bestätigte\s*Coronafälle\s*in\s*der\s*StädteRegion\s*" + \
    r"(?:Aachen)?\s*[(]davon\s*([0-9]{1,6})\s*in\s*der\s*Stadt\s*Aachen[)].*"
    pattern_recovered = r"([0-9]{1,6})\s*ehemals\s*positiv\s*auf\s*das\s*Corona-Virus\s*getestete\s*" + \
    r"Personen\s*sind\s*inzwischen\s*wieder\s*gesund.*"
    pattern_deaths = r"Bislang\s*([0-9]{1,6})\s*Todesfälle.*"
    pattern_active = r"Damit\s*aktuell\s*([0-9]{1,6})\s*nachgewiesene?\s*Infizie.*"
        
    return parse_cases("Abstract", abstract, pattern_total, pattern_recovered, pattern_deaths, pattern_active)

def parse_main(main: str) -> CaseFigures:
    """
    Analyse <main> text of the news and return Covid-19 case numbers included in main text. 
    
    The relevant area of the <abstract> is expected to have the following format:
      
    'Es gibt insgesamt in der StädteRegion [nunmehr] 1997 positive Fälle, davon 992 in der Stadt Aachen'
    '1876 ehemals positiv auf das Corona-Virus getestete Personen sind inzwischen wieder gesund'
    'Die Zahl der gemeldeten Todesfälle liegt [nach wie vor] bei 98'
    'Damit sind aktuell 23 Menschen in der StädteRegion nachgewiesen infiziert'

    Parameters:
    ===========
    
    Returns:
    ========
    None - if abstract could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
    """
    log.debug("parse_main(" + main[:40] + "...)")

    pattern_total = r"Es\s*gibt\s*insgesamt\s*in\s*der\s*StädteRegion\s*(?:nunmehr)?\s*([0-9]{1,6})\s*" + \
    r"positive\s*Fälle,\s*davon\s*([0-9]{1,6})\s*in\s*der\s*Stadt\s*Aachen"
    pattern_recovered = r"([0-9]{1,6})\s*ehemals\s*positiv\s*auf\s*das\s*Corona-Virus\s*getestete\s*" + \
    r"Personen\s*sind\s*inzwischen\s*wieder\s*gesund"
    pattern_deaths = r"Die\s*Zahl\s*der\s*gemeldeten\s*Todesfälle\s*liegt\s*(?:nach\s*wie\s*vor)?\s*" + \
    r"bei\s*([0-9]{1,6})"
    pattern_active = r"Damit\s*sind\s*aktuell\s*([0-9]{1,6})\s*Menschen\s*in\s*der\s*StädteRegion\s*" + \
    r"(?:Aachen)?\s*nachgewiesen\s*infiziert"
    
    return parse_cases("Main", main, pattern_total, pattern_recovered, pattern_deaths, pattern_active)

def checked_data(meta: NewsMeta, fig_abstract: CaseFigures, fig_main: CaseFigures) -> []:
    """
    Check extracted data for consistency and return valid figures or None.
    
    Parameters:
    ===========
    
    Returns:
    ========
    [ Date, Time, Total, Total_AC, Deaths, Recovered, Active ] - Array w/ read Covid-19 numbers
    None - if consistency check fails
    """
    
    if meta != None: 
        # Der Header konnte erfolgreich analysiert werden
        a, m = fig_abstract, fig_main
        if (a != None) and (m != None): 
            # Sowohl Abstract als auch Main konnten erfolgreich analysiert werden
            if (m.total == a.total) and (m.deaths == a.deaths) and (m.recovered == a.recovered) and \
               (m.active == a.active):
                # Die eingelesenen Zahlen aus Abstract und Main stimmmen überein
                if m.active == 0:
                    # Die Anzahl aktiver Fälle war nicht explizit angegeben und wird stattdessen berechnet
                    m.active = m.total - m.deaths - m.recovered
                if m.total - m.deaths - m.recovered == m.active:
                    # Die eingelesenen Daten haben die Konsistenzprüfung bestanden
                    return [ meta.date, meta.time, m.total, m.total_AC, m.deaths, m.recovered, m.active ]
                else:
                    log.warning('Data consistency error: (1) Main and Abstract found. Inconsistent figures read.')
                    return None
        elif m != None: 
            # Mindestens Main konnte erfolgreich analysiert werden
            if m.active == 0:
                # Die Anzahl aktiver Fälle war nicht explizit angegeben und wird stattdessen berechnet
                m.active = m.total - m.deaths - m.recovered
            if m.total - m.deaths - m.recovered == m.active:
                # Die eingelesenen Daten haben die Konsistenzprüfung bestanden
                return [ meta.date, meta.time, m.total, m.total_AC, m.deaths, m.recovered, m.active ]
            else:
                log.warning('Data onsistency error: (2) Only Main found. Inconsistent figures read. ')
                return None
    else:
        log.warning('Data consistency error: (3) Header not found.')
        return None

def fill_dataframe_from_text(text_records: str) -> pd.DataFrame:
    """
    Analyse <text_records> and return Pandas dataframe with parsed Covid-19 case numbers. 
    
    Parameters:
    ===========
    text_records
    
    Returns:
    ========
    pd.DataFrame - 
    """
    
    log.debug(f"fill_dataframe_from_text(%d records)", len(text_records))

    figures = [] # Zwischenspeicherung der eingelesenen Zahlen
    dates = [] # Zwischenspeicherung der Datumsangaben
    
    for tr in text_records:
        meta = parse_header(tr['Header'])
        fig_abstract = parse_abstract(tr['Abstract'])
        fig_main = parse_main(tr['Main'])
        
        rec = checked_data(meta, fig_abstract, fig_main)
        if rec:
            dates.append(rec[0])
            figures.append(rec[1:])
            

    if figures.count:
        cols = ['Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]
        index = pd.DatetimeIndex(dates, name='Datum')
        df = pd.DataFrame(np.array(figures), columns=cols, index=index).sort_index()
        return df
    
    return pd.DataFrame() # Leer

## Datenabfrage durchführen

1. Datum des letzten Datensatzes in der Excel-Datei ermitteln und mit aktuellem Datum vergleichen. 
2. Wenn mindestens 1 Tag seit der letzten Datenabfrage vergangen ist, neue Abfrage durchführen.
3. Von der Website gelieferte Rohtexte auswerten und Fallzahlen extrahieren
4. Extrahierte Fallzahlen auf Konsistenz prüfen
5. Geprüfte Fallzahlen zu neuem DataFrame hinzufügen
6. Den neuen mit dem existierendem DataFrame zusammenführen
7. Zusammengeführte Daten speichern

In [28]:
new_request = (c19_cases.size == 0) or \
    (date.today() >= (date.fromtimestamp(c19_cases.index.max().timestamp()) + timedelta(days=1)))

if new_request:
    log.info("New request initiated")
    if not html_payload:
        url = config['Rohdaten']['SourceURLDefault']
        if robot_access_allowed(url):
            html_payload = gather_html(url)

    if html_payload:
        records = gather_text(html_payload)

    if records.count:
        new_cases = fill_dataframe_from_text(records)

    if not new_cases.empty:
        # Nur Zeilen mit neuerem Datum hinzufügen
        merged_cases = pd.concat([c19_cases, new_cases[new_cases.index > c19_cases.index[-1]]], join='outer')

    merged_cases.to_excel(config['Rohdaten']['FileName'], 
                          sheet_name=config['Rohdaten']['SheetName'], index_label='Datum')
    
    merged_cases.tail(10)
else:
    log.info("No new request required")